In [8]:
import numpy as np
from quantrocket import get_prices

def ViterbiOptimalIndex(states, init, trans, emit, obs):
    S = len(states)
    T = len(obs)
    
    prob = np.zeros((T, S))
    prev = np.zeros((T, S), dtype=int)
    portfolio_value = 0
    optimal_buy_indices = []
    
    # Initialization
    for s in range(S):
        prob[0][s] = init[s] * emit[s][obs[0]]
    
    # Forward algorithm
    for t in range(1, T):
        for s in range(S):
            max_prob = 0
            max_state = -1
            for r in range(S):
                new_prob = prob[t - 1][r] * trans[r][s] * emit[s][obs[t]]
                if new_prob > max_prob:
                    max_prob = new_prob
                    max_state = r
            prob[t][s] = max_prob
            prev[t][s] = max_state
    
    # Backward algorithm to find optimal path
    max_prob_last_state = 0
    max_state_last_state = -1
    for s in range(S):
        if prob[T - 1][s] > max_prob_last_state:
            max_prob_last_state = prob[T - 1][s]
            max_state_last_state = s
    
    path = np.zeros(T, dtype=int)
    path[T - 1] = max_state_last_state
    for t in range(T - 1, 0, -1):
        path[t - 1] = prev[t][path[t]]
    
    # Portfolio optimization
    state_transitions = np.zeros((S, S))
    for i in range(1, T):
        if path[i] == 1 and path[i - 1] == 0 and trans[1][2] > trans[1][0]:
            portfolio_value += 1
            optimal_buy_indices.append(i - 1)
            state_transitions[path[i - 1]][path[i]] += 1
        elif path[i] == -1 and path[i - 1] == 0 and trans[0][1] > trans[0][2]:
            portfolio_value -= 1
            state_transitions[path[i - 1]][path[i]] += 1
        else:
            state_transitions[path[i - 1]][path[i]] += 1
    
    # Calculate transition distribution
    transition_distribution = np.zeros((S, S))
    for i in range(S):
        total_sum = np.sum(state_transitions[i])
        for j in range(S):
            transition_distribution[i][j] = state_transitions[i][j] / total_sum
    
    return portfolio_value, optimal_buy_indices, transition_distribution
